In [1]:
import os
import cv2
from pathlib import Path
from tqdm import tqdm

import lxml 
import xml.etree.ElementTree 

In [18]:
classes = {'ao-xian': 0}

In [1]:
path = Path('VOCdevkit/VOC2007/Annotations/')

In [45]:
xml_files = os.listdir(str(path))

In [49]:
for file in tqdm(xml_files):
    
    file_path = path / file
    
    if file_path.suffix == '.xml':
        
        xmltree = xml.etree.ElementTree.parse(
            
            str(file_path.absolute()), 
                                              
            parser=lxml.etree.XMLParser(encoding='utf-8')
                                             
        ).getroot()
        
        filename = xmltree.find('filename').text
        
        img_src_path = path.parent / 'JPEGImages' / filename

        for object_iter in xmltree.findall('object'):
            
            bndbox = object_iter.find("bndbox")
            
            label = object_iter.find('name').text

            difficult = False
            if object_iter.find('difficult') is not None:
                difficult = bool(int(object_iter.find('difficult').text))

            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            src_path = path.parent / 'labels_yolo' / (file.split('.')[0] + '.txt')
            
            with open(str(src_path.absolute()), 'w', encoding='utf-8') as f:
                
                class_idx = classes[label]
                
                height, width, _ = cv2.imread(str(img_src_path.absolute())).shape

                center_x = (xmin + xmax) / 2 / width
                center_y = (ymin + ymax) / 2 / height
                w = (xmax - xmin) / width
                h = (ymax - ymin) / height
                
                center_x = round(center_x, 6)
                center_y = round(center_y, 6)
                w = round(w, 6)
                h = round(h, 6)
                
                s = f'{class_idx}, {center_x}, {center_y}, {w}, {h}\n'
                
                f.write(s)

100%|██████████| 169/169 [00:30<00:00,  5.48it/s]
